In [12]:
pandas_df = promedios.toPandas()
pandas_df.head()


,city,parameter,avg_value,n_registros
0,Medellín,PM2.5,42.0,1
1,Barranquilla,CO2,430.0,1
2,Cali,PM2.5,29.0,1
3,Barranquilla,PM10,60.0,1
4,Bogotá,PM2.5,36.0,1


In [11]:
output_path = "/content/drive/MyDrive/Tarea3_Spark/resultados_promedios"
promedios.coalesce(1).write.mode("overwrite").option("header",True).csv(output_path)


In [10]:
from pyspark.sql.functions import avg, count

promedios = df_clean.groupBy("city","parameter") \
    .agg(avg("value").alias("avg_value"), count("*").alias("n_registros"))

promedios.show(10, truncate=False)


+------------+---------+---------+-----------+
|city        |parameter|avg_value|n_registros|
+------------+---------+---------+-----------+
|Medellín    |PM2.5    |42.0     |1          |
|Barranquilla|CO2      |430.0    |1          |
|Cali        |PM2.5    |29.0     |1          |
|Barranquilla|PM10     |60.0     |1          |
|Bogotá      |PM2.5    |36.0     |1          |
|Bogotá      |PM10     |54.0     |1          |
|Cartagena   |PM2.5    |33.0     |1          |
|Cali        |NO2      |18.0     |1          |
|Medellín    |CO2      |410.0    |1          |
|Cartagena   |NO2      |22.0     |1          |
+------------+---------+---------+-----------+



In [9]:
df_clean = df.dropna(subset=["city","parameter","value"])
from pyspark.sql.functions import to_timestamp, col
df_clean = df_clean.withColumn("date_utc", to_timestamp(col("date_utc")))
df_clean.printSchema()


root
 |-- city: string (nullable = true)
 |-- parameter: string (nullable = true)
 |-- value: integer (nullable = true)
 |-- unit: string (nullable = true)
 |-- date_utc: timestamp (nullable = true)



In [8]:

df = spark.read.csv("calidad_aire.csv", header=True, inferSchema=True)

df.printSchema()
df.show(5)


root
 |-- city: string (nullable = true)
 |-- parameter: string (nullable = true)
 |-- value: integer (nullable = true)
 |-- unit: string (nullable = true)
 |-- date_utc: timestamp (nullable = true)

+--------+---------+-----+-----+-------------------+
|    city|parameter|value| unit|           date_utc|
+--------+---------+-----+-----+-------------------+
|  Bogotá|    PM2.5|   36|µg/m³|2025-10-15 10:00:00|
|  Bogotá|     PM10|   54|µg/m³|2025-10-15 10:00:00|
|Medellín|    PM2.5|   42|µg/m³|2025-10-15 10:05:00|
|Medellín|      CO2|  410|  ppm|2025-10-15 10:05:00|
|    Cali|    PM2.5|   29|µg/m³|2025-10-15 10:10:00|
+--------+---------+-----+-----+-------------------+
only showing top 5 rows



In [7]:

data = """city,parameter,value,unit,date_utc
Bogotá,PM2.5,36,µg/m³,2025-10-15 10:00:00
Bogotá,PM10,54,µg/m³,2025-10-15 10:00:00
Medellín,PM2.5,42,µg/m³,2025-10-15 10:05:00
Medellín,CO2,410,ppm,2025-10-15 10:05:00
Cali,PM2.5,29,µg/m³,2025-10-15 10:10:00
Cali,NO2,18,µg/m³,2025-10-15 10:10:00
Barranquilla,PM10,60,µg/m³,2025-10-15 10:15:00
Barranquilla,CO2,430,ppm,2025-10-15 10:15:00
Cartagena,PM2.5,33,µg/m³,2025-10-15 10:20:00
Cartagena,NO2,22,µg/m³,2025-10-15 10:20:00
"""

with open("calidad_aire.csv", "w") as f:
    f.write(data)

print("✅ Archivo 'calidad_aire.csv' creado correctamente.")


✅ Archivo 'calidad_aire.csv' creado correctamente.


In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Tarea3_CalidadAire") \
    .master("local[*]") \
    .config("spark.ui.port", "4041") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")
print(spark)


In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [1]:
!apt-get update -q
!apt-get install -y openjdk-8-jdk-headless -qq
!pip install -q pyspark


Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 https://cli.github.com/packages stable InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,372 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [5,803 kB]
Get:14 http://